In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import os
import sys
import jax
import jax.numpy as jnp
import jax.random
from gymnasium import spaces
from tqdm import tqdm

from klax import lipschitz_l1_jax, triangular
from rl_environments import Vandelpol
# from rl_environments import LDSEnv, InvertedPendulum, CollisionAvoidanceEnv, Vandelpol
from rsm_learner import Learner
from rsm_verifier import Verifier
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

2025-01-06 19:36:15.133123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-06 19:36:15.173785: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-06 19:36:15.186514: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
parser = argparse.ArgumentParser(
    prog='System Normalization for Neural Certificates',
    description='Learning a reach-avoid supermartingale neural network with normalization techniques',
    epilog='By Musan (Hao Wu) @ SKLCS, Institute of Software, UCAS'
)
# problem formulation    
parser.add_argument("--env", default="vandelpol", help='control system')
parser.add_argument("--timeout", default=60, type=int, help='max time limit in minutes') 
parser.add_argument("--reach_prob", default=0.8, type=float, help='reach-avoid probability')

# neural network and training
parser.add_argument("--hidden", default=128, type=int, help='hidden neurons in each layer')
parser.add_argument("--num_layers", default=2, type=int, help='number of hidden layers')
# learner 
parser.add_argument("--continue_rsm", type=int, default=0, help='use an existing network')

# verifier
parser.add_argument("--eps", default=0.05, type=float) # ???
parser.add_argument("--lip", default=0.01, type=float) # ???
# parser.add_argument("--p_lip", default=0.0, type=float) 
parser.add_argument("--l_lip", default=4.0, type=float) # ???
parser.add_argument("--fail_check_fast", type=int, default=0)
parser.add_argument("--grid_factor", default=1.0, type=float)
parser.add_argument("--batch_size", default=512, type=int)
# parser.add_argument("--ppo_iters", default=50, type=int)
# parser.add_argument("--policy", default="policies/lds0_zero.jax")
# parser.add_argument("--train_p", type=int, default=1)
parser.add_argument("--square_l_output", default=True)
parser.add_argument("--jitter_grid", type=int, default=0)
parser.add_argument("--soft_constraint", type=int, default=1)
parser.add_argument("--gamma_decrease", default=1.0, type=float)
parser.add_argument("--debug_k0", action="store_true")
parser.add_argument("--gen_plot", action="store_true")
parser.add_argument("--no_refinement", action="store_true")
parser.add_argument("--plot", action="store_true")
parser.add_argument("--small_mem", action="store_true")
sys.argv = ['test.ipynb', '--env', 'vandelpol']
args = parser.parse_args()

In [4]:
env = Vandelpol()
learner = Learner(
    env=env,
    l_hidden=[args.hidden] * args.num_layers,
    l_lip=args.l_lip,
    eps=args.eps,
    gamma_decrease=args.gamma_decrease,
    reach_prob=args.reach_prob,
    softplus_l_output=args.square_l_output,
)

/home/wuhao/micromamba/envs/nc/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [5]:
verifier = Verifier(
    learner=learner,
    env=env,
    batch_size=args.batch_size,
    reach_prob=args.reach_prob,
    fail_check_fast=bool(args.fail_check_fast),
    grid_factor=args.grid_factor,
    small_mem=args.small_mem,
)

In [6]:
from rsm_loop import RSMLoop
loop = RSMLoop( 
        learner,
        verifier,
        env,
        lip_factor=args.lip,
        plot=args.plot,
        # train_p=bool(args.train_p),#???
        jitter_grid=bool(args.jitter_grid),#???
        soft_constraint=bool(args.soft_constraint),#???
    )


# txt_return = learner.evaluate_rl()
loop.plot_l(f"plots/{args.env}_start.png")

In [7]:
loop.run(args.timeout*10)


## Iteration 0 (0:01 elapsed) ##
use train buffer.


Train: loss=0.687, dec_loss=0.355, violations=0.539: 100%|██████████| 100/100 [00:49<00:00,  2.03it/s]


Trained on 40000 samples, start_loss=7.4, end_loss=0.687, start_violations=0.542, end_violations=0.539 in 49.3 seconds


AttributeError: 'list' object has no attribute 'low'

In [9]:
init = env.init_spaces[0]

In [10]:
init.low

array([-0.2,  0.2], dtype=float32)

In [13]:
def v_contains(boxs, states):
    '''
    Check if a vector of states is contained in any of the boxes
    Return:
        contains: a boolean vector of length states.shape[0]
    '''
    contains = np.zeros(states.shape[0], dtype=jnp.bool)
    for box in boxs:
        b_low = np.expand_dims(box.low, axis=0)
        b_high = np.expand_dims(box.high, axis=0)
        mask = np.logical_and(
            np.all(states >= b_low, axis=1), np.all(states <= b_high, axis=1)
        )
        contains = np.logical_or(mask, contains)
    return contains

b = [spaces.Box(
            low = np.array([-2.0, -2.0], dtype=jnp.float32),  
            high = np.array([2.0, 2.0], dtype=jnp.float32),
            dtype=jnp.float32)]
s = jnp.array([[-1.0, 1.0], [3.0, 1.0]])

v_contains(b, s)

TypeError: 'Box' object is not iterable